Note: Select "Run" above to run all cells, or press "shift + enter"

First we need to install required packages. Might take a minute or two:

In [ ]:
!pip install -q numpy matplotlib scipy ipywidgets sycomore

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

import sycomore
from sycomore import units

Suppose a spin isochromat has magnetization $M_0$ and resonance offset $\Delta \omega = \omega - \omega_{ref}$, where $\omega_{ref}$ is the angular frequency of the rotating frame. 

Immediately after an excitation pulse (flip angle = $\theta_1$) along the x-axis, the magnetization of the spin isochromat can be described by:

$$\vec{M}(\Delta \omega, t = 0) = M_0 \begin{bmatrix} 0 \\ \sin \theta_1 \\ \sin \theta_2 \end{bmatrix}$$

In general, $\vec{M}$ can have a different Larmor frequency $\omega$ than the rotating frame $\omega_{ref}$, leading to non-zero values of $\Delta \omega$. Because of this resonance offset, $\vec{M}$ precesses about the z-axis as described by:

$$\vec{M}(\Delta \omega, t) = M_0 \begin{bmatrix} 
e^{-t/T_2}\sin\theta_1\sin(\Delta \omega t)  \\ 
e^{-t/T_2}\sin\theta_1\cos(\Delta \omega t)  \\ 
e^{-t/T_1}\cos\theta_1 + (1-e^{-t/T_1}) \end{bmatrix}$$


where the exponential terms account for the $T_1$ and $T_2$ relaxation effects. 

Let's implement this expression. Below, `dw` is the angular frequency shift $\Delta \omega = 2\pi \Delta f$.

In [338]:
def calculate_magnetization(time_ms, species, flip_angle):
    
    # Note: the sycomore packages defines a counterclockwise rotation for positive angles, whereas 
    # Bernstein defined a clockwise rotation for positive angles. Therefore, we need to invert the rotation here. 
    # ref: https://github.com/lamyj/sycomore/blob/master/src/sycomore/isochromat/Model.cpp
    t = np.array(time_ms)

    dw =  - 2 * np.pi * species.delta_omega.convert_to(units.Hz)
    
    E1 = np.exp(-t/species.T1.convert_to(units.ms))
    E2 = np.exp(-t/species.T2.convert_to(units.ms))

    Mx = E2 * np.sin(flip_angle.convert_to(units.rad)) * np.sin(dw*t*1e-3)
    My = E2 * np.sin(flip_angle.convert_to(units.rad)) * np.cos(dw*t*1e-3)
    Mz = E1 * np.cos(flip_angle.convert_to(units.rad)) + (1 - E1)

    return np.stack((Mx, My, Mz), -1)

We will use the expression defined in Bernstein for a frequency shift:


$$ f = \frac{\gamma}{2\pi}B_0(1-\delta)$$

Therefore, 

$$ \Delta f = f_{rot} -f $$ 


Let's implement this expression. Keep in mind $\frac{\gamma}{2\pi} = 42.58\times 10^6$ Hz.

In [339]:
def get_inhomogeneity(B0=3, inhomogeneity_ppm=0.0):

    # Define the gyromagnetic ratio for protons (42.58 MHz/T)
    gyromagnetic_ratio = 42.58e6  # Hz/T

    freq_rot = gyromagnetic_ratio * B0 # resonant frequency 
    freq = gyromagnetic_ratio * B0 * (1 - inhomogeneity_ppm) 

    delta_freq = (freq - freq_rot)*1e-6 # convert from ppm

    return delta_freq*units.Hz 

Below are some convenient functions to define the species of the spin isochromat as well as to modifty/record the magnetization state 

In [341]:
def get_species(T1, T2, delta_omega=0*units.Hz):   
    species = sycomore.Species(T1, T2, delta_omega=delta_omega)
    return species

def update_and_record_magnetization(M, record, t, update=None):
    if update is not None: M = update @ M
    record.append([t.convert_to(units.ms), M[:3] / M[3]])
    return M

Let's simulate a simple excitation-recovery pulse and record the spin behavior.

In [352]:


def run_simulation(T1_ms, T2_ms, flip_angle_deg, step_size_ms):
    
    # define species of spin isochromat
    # ---------------------------------

    delta_omega = get_inhomogeneity(B0=3, inhomogeneity_ppm=0.2)

    T1 = T1_ms * units.ms
    T2 = T2_ms * units.ms

    species = get_species(T1, T2, delta_omega=-delta_omega)

    # initialize the spin state 
    t = 0 * units.ms
    M = np.array([0, 0, 1, 1])
    record = [[t.convert_to(units.ms), M[:3] / M[3]]]

    # define experiment bloacks
    # -------------------------

    # temporal resolution of experiment 
    step_size = step_size_ms * units.ms 

    # nothing occurs during this time
    idle = sycomore.bloch.time_interval(species, step_size) 

    # excitation pulse, phase implies it is along x-axis. Vector rotates about x-axis. If flip angle is 90 degrees, vector flip to +y-axis
    flip_angle = flip_angle_deg * units.deg
    pulse = sycomore.bloch.pulse(flip_angle, phase=np.pi*units.rad) 

    # define experiment blocks
    # ------------------------

    # apply pulse
    M = update_and_record_magnetization(M, record, t, update=pulse)

    # run 100 steps 
    for _ in range(100):
        t += step_size
        M = update_and_record_magnetization(M, record, t, update=idle)

    time, magnetization = list(zip(*record))

    magnetization = np.array(magnetization).round(3)

    magnetization_predicted = calculate_magnetization(time, species, flip_angle)

    plt.figure(figsize=(10,5))
    plt.subplot(121)
    plt.plot(time, magnetization[:, 0], label="Species $M_x$", color='#6AD991')
    plt.plot(time, magnetization_predicted[:,0], label="Species $M_x$ Predicted", linestyle='--', color='#F25050')
    plt.plot(time, magnetization[:, 2], label="Species $M_z$", linestyle='dotted', color='#BF1515')
    plt.plot(time, magnetization_predicted[:,2], label="Species $M_z$ Predicted", linestyle='dashdot', color='#F2668B')
    plt.plot(time, np.linalg.norm(magnetization[:, :2], axis=-1), label="Species $M_\perp$", color='black')
    plt.ylim(-1,1)
    plt.legend()

    plt.subplot(122)
    plt.plot(time, magnetization[:, 1], label="Species $M_y$", color='#6AD991')
    plt.plot(time, magnetization_predicted[:,1], label="Species $M_y$ Predicted", linestyle='--', color='#F25050')
    plt.plot(time, np.linalg.norm(magnetization[:, :2], axis=-1), label="Species $M_\perp$", color='black')
    plt.ylim(-1,1)
    plt.legend()

    plt.tight_layout()
    plt.show()


In [353]:
widgets.interact(
    run_simulation,
    T1_ms=widgets.FloatSlider(min=100, max=2000, step=10, value=1000, description='T1 (ms)'),
    T2_ms=widgets.FloatSlider(min=10, max=200, step=1, value=100, description='T2 (ms)'),
    flip_angle_deg=widgets.FloatSlider(min=0, max=180, step=1, value=90, description='Flip angle (deg)'),
    step_size_ms=widgets.FloatSlider(min=0.1, max=10, step=0.1, value=1, description='Step size (ms)')
)

interactive(children=(FloatSlider(value=1000.0, description='T1 (ms)', max=2000.0, min=100.0, step=10.0), Floa…

<function __main__.run_simulation(T1_ms, T2_ms, flip_angle_deg, step_size_ms)>

In [354]:
pwd

'/mnt/alp/Users/Manuel/manuscripts/LGE/code'

In [ ]:

delta_omega = get_inhomogeneity(B0=3, inhomogeneity_ppm=1)

species_A = get_species(T1=1000 * units.ms, T2=100 * units.ms, delta_omega=delta_omega)
species_B = get_species(T1=1000 * units.ms, T2=100 * units.ms, delta_omega=-delta_omega)

flip_angle = 90 * units.deg

idle_A = sycomore.bloch.time_interval(species_A, 10 * units.ms)
idle_B = sycomore.bloch.time_interval(species_B, 10 * units.ms)
pulse = sycomore.bloch.pulse(flip_angle)

# initialize the spins
t = 0 * units.s

M_A = np.array([0, 0, 1, 1])
M_B = np.array([0, 0, 1, 1])
M_C = M_A + M_B

record_A = [[t.convert_to(units.ms), M_A[:3] / M_A[3]]]
record_B = [[t.convert_to(units.ms), M_B[:3] / M_B[3]]]
record_C = [[t.convert_to(units.ms), M_C[:3] / M_C[3]]]

# Update and record magnetization for the first 10 steps
for _ in range(10):
    t += 10 * units.ms
    M_A = update_and_record_magnetization(M_A, idle_A, record_A, t)
    M_B = update_and_record_magnetization(M_B, idle_B, record_B, t)
    M_C = update_and_record_magnetization(M_A + M_B, None, record_C, t)
    
# Apply the pulse and record the magnetization
M_A = update_and_record_magnetization(M_A, pulse, record_A, t)
M_B = update_and_record_magnetization(M_B, pulse, record_B, t)
M_C = update_and_record_magnetization(M_A + M_B, None, record_C, t)

# Update and record magnetization for the next 100 steps
for _ in range(100):
    t += 10 * units.ms
    
    M_A = update_and_record_magnetization(M_A, idle_A, record_A, t)
    M_B = update_and_record_magnetization(M_B, idle_B, record_B, t)
    M_C = update_and_record_magnetization(M_A + M_B, None, record_C, t)
